In [1]:
# !pip install transformers datasets scikit-learn
!pip install torch transformers


Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [3]:
# import torch
# from transformers import LlamaTokenizer, LlamaForSequenceClassification, Trainer, TrainingArguments
# from datasets import Dataset
# from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# # Sample dataset creation
# data = {
#     'context': [
#         "JSX stands for JavaScript XML. It allows us to write HTML in React. JSX makes it easier to write and add HTML in React.",  # Easy
#         "A React component is a reusable piece of UI. It can be a class-based or functional component, and it returns JSX that defines how the UI should look.",  # Easy
#         "useState is a React hook that allows functional components to have local state. It takes the initial state as an argument and returns an array with two values: the current state and a function to update the state.",  # Medium
#         "The Context API allows you to share global state across components without having to pass props down manually at every level. It consists of a Provider and a Consumer.",  # Medium
#         "React's rendering process involves a virtual DOM. When the state of an object changes, React first changes the object in the virtual DOM, then it compares it with the previous version and updates the actual DOM accordingly.",  # Hard
#         "React Router is a standard library for routing in React. It enables the navigation among views of various components in a React Application, allows changing the browser URL, and keeps UI in sync with the URL."  # Hard
#     ],
#     'label': [0, 0, 1, 1, 2, 2]  # 0: Easy, 1: Medium, 2: Hard
# }

# dataset = Dataset.from_dict(data)

# train_test_split = dataset.train_test_split(test_size=0.2)
# train_dataset = train_test_split['train']
# test_dataset = train_test_split['test']

# # Load Llama tokenizer and model
# tokenizer = LlamaTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf')
# model = LlamaForSequenceClassification.from_pretrained('meta-llama/Llama-2-7b-hf', num_labels=3)

# # Tokenize data
# def tokenize(batch):
#     return tokenizer(batch['context'], padding=True, truncation=True)

# train_dataset = train_dataset.map(tokenize, batched=True)
# test_dataset = test_dataset.map(tokenize, batched=True)

# train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
# test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

# # Compute metrics
# def compute_metrics(p):
#     preds = p.predictions.argmax(-1)
#     precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='weighted')
#     acc = accuracy_score(p.label_ids, preds)
#     return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# # Training arguments
# training_args = TrainingArguments(
#     output_dir='./results',
#     evaluation_strategy="epoch",
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=3,
#     weight_decay=0.01,
# )

# # Initialize Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset,
#     compute_metrics=compute_metrics
# )

# # Train the model
# trainer.train()

# # Prediction function
# def predict_difficulty(context):
#     inputs = tokenizer(context, return_tensors='pt', padding=True, truncation=True)
#     with torch.no_grad():
#         outputs = model(**inputs)
#     logits = outputs.logits
#     predicted_class_id = logits.argmax().item()
#     difficulty = {0: "easy", 1: "medium", 2: "hard"}
#     return difficulty[predicted_class_id]

# # Test the model with new context
# new_context = "Memoization is an optimization technique used in React to prevent unnecessary re-renders of components. By wrapping components with React.memo, developers can create memoized components that only re-render when their props change. This is particularly useful for performance-sensitive applications where components receive complex data structures or large lists. Memoization helps maintain application responsiveness and can lead to a smoother user experience by minimizing the rendering workload."
# print(f"Predicted difficulty: {predict_difficulty(new_context)}")




import torch
from transformers import LlamaTokenizer, LlamaForCausalLM


model_path = 'meta-llama/Llama-2-7b-hf'  
tokenizer = LlamaTokenizer.from_pretrained(model_path)
model = LlamaForCausalLM.from_pretrained(model_path)


def create_prompt(context):
    prompt = f"Classify the difficulty level of the following concept as 'easy', 'medium', or 'hard':\n\n{context}\n\nDifficulty level:"
    return prompt

def predict_difficulty(context):
    prompt = create_prompt(context)
    inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True)
    

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=100, num_return_sequences=1)

    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    
    if 'easy' in output_text.lower():
        return "easy"
    elif 'medium' in output_text.lower():
        return "medium"
    elif 'hard' in output_text.lower():
        return "hard"
    else:
        return "unknown"


new_context = "Memoization is an optimization technique used in React to prevent unnecessary re-renders of components. By wrapping components with React.memo, developers can create memoized components that only re-render when their props change. This is particularly useful for performance-sensitive applications where components receive complex data structures or large lists. Memoization helps maintain application responsiveness and can lead to a smoother user experience by minimizing the rendering workload."

predicted_difficulty = predict_difficulty(new_context)
print(f"Predicted difficulty: {predicted_difficulty}")



OSError: Can't load tokenizer for 'meta-llama/Llama-2-7b-hf'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'meta-llama/Llama-2-7b-hf' is the correct path to a directory containing all relevant files for a LlamaTokenizer tokenizer.